# Parse property tax records

In [1]:
import re
import csv
import settings
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
class SBAssessorParse(object):
    """
    A base parser for extracting data from Santa Barbara County Assessor pages.
    """
    def __init__(self, apn, year=2018):
        self.apn = apn
        self.year = year
    
    def parse(self):
        raise NotImplementedError
   
    @property
    def inpath(self):
        return f"{settings.input_dir}/scrape/{self.slug}-{self.apn}-{self.year}.html"

    def get_soup(self):
        with open(self.inpath, 'rb') as f:
            data = f.read() 
        return BeautifulSoup(data, "html.parser")
    
    def safemoney(self, s):
        return float(s.strip().replace("$", "").replace(",", ""))

In [3]:
class DetailsParse(SBAssessorParse):
    """
    Parse data from APN's detail page.
    """
    slug = 'details'

    def parse(self):
        soup = self.get_soup()
        use = soup.find(text=re.compile("Use Description")).parent.parent.parent.find_all("td")[2].text.strip()
        details_net_assessed_value = soup.find(text=re.compile("Net Assessed Value")).parent.parent.parent.find_all("td")[2].text
        return {
            "apn": self.apn,
            "use": use,
            "details_net_assessed_value": self.safemoney(details_net_assessed_value),
        }

In [4]:
class ValueNoticeParse(SBAssessorParse):
    """
    Parse data from APN's valuenotice page.
    """
    slug = "valuenotice"
    
    def parse(self):
        soup = self.get_soup()
        total_values = soup.find(text=re.compile("Total Assessed Value")).parent.parent.find_all("td")
        net_values = soup.find(text=re.compile("Net Assessed Value")).parent.parent.find_all("td")
        if len(total_values) == 4:
            data = {
                "apn": self.apn,
                "valuenotice_economic_value": self.safemoney(total_values[1].text),
                "valuenotice_prop13_value": self.safemoney(total_values[2].text),
                "valuenotice_total_taxable_value": self.safemoney(total_values[3].text),
            }
        elif len(total_values) == 2:
            data = {
                "apn": self.apn,
                "valuenotice_economic_value": None,
                "valuenotice_prop13_value": self.safemoney(total_values[1].text),
                "valuenotice_total_taxable_value": self.safemoney(total_values[1].text),
            }
        if net_values:
            data["valuenotice_net_taxable_value"] = self.safemoney(net_values[-1].text)
        return data

In [5]:
class BillParse(SBAssessorParse):
    """
    Parse data from APN's valuenotice page.
    """
    slug = 'bill'

    def parse(self):
        soup = self.get_soup()
        bill_total_tax = soup.find(text=re.compile("TOTAL TAX")).parent.parent.parent.parent.parent.parent.find_all("td")[2].text
        return {
            "apn": self.apn,
            "bill_total_tax": self.safemoney(bill_total_tax)
        }

Read in all the APNs

In [6]:
apn_list = [r['apn'] for r in list(csv.DictReader(open(f"{settings.output_dir}/hollister-parcels.csv")))]

Run the parsers

In [7]:
detail_data = pd.DataFrame([DetailsParse(a).parse() for a in apn_list])

In [8]:
valuenotice_data = pd.DataFrame([ValueNoticeParse(a).parse() for a in apn_list])

In [9]:
bill_data = pd.DataFrame([BillParse(a).parse() for a in apn_list])

Merge them

In [10]:
merged_df = bill_data.merge(
    pd.merge(
        detail_data,
        valuenotice_data,
        on="apn",
        how="inner"
    ),
    on="apn",
    how="inner"
)

Filter out any that didn't get a Williamson Act assessment

In [11]:
merged_df = merged_df[~pd.isnull(merged_df.valuenotice_economic_value)]

In [12]:
cleaned_df = merged_df.rename(columns={
    "valuenotice_total_taxable_value": "williamson_assessment",
    "valuenotice_prop13_value": "prop13_assessment",
    "bill_total_tax": "williamson_tax",
})

In [13]:
trimmed_df = cleaned_df[[
    'apn',
    'williamson_assessment',
    'prop13_assessment',
    'williamson_tax'
]]

Merge it with the shapefile

In [14]:
parcels_df = pd.read_csv(f"{settings.output_dir}/hollister-parcels.csv", dtype={"apn": str})

In [15]:
analysis_df = parcels_df.merge(trimmed_df, on="apn", how="inner")

Output

In [16]:
analysis_df.to_csv(f"{settings.output_dir}/parsed.csv", index=False, encoding="utf-8")